In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emirhan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
from tensorflow.keras.models import Sequential
import pickle
import random
import json


In [3]:
#read json file
with open('bot.json' , encoding='utf-8') as f:
    data=json.load(f)
    print(data['intents'][0])

{'tag': 'greeting', 'patterns': ['Merhaba', 'Nasılsın', 'Selam', 'Sa', 'İyi günler'], 'responses': ['Merhaba, teşekkürler', 'Seni yeniden görmek güzel', 'Selam, nasıl yardımcı olabilirim?'], 'context': ['']}


In [4]:
#tokenization
words=[]
classes=[]
documents=[]
ignore_words=['?']
#get into  json
for intent in data['intents']:
    for pattern in intent['patterns']:
        #tokenize each word in the sentence
        w=nltk.word_tokenize(pattern)
        #add words into list
        words.extend(w)#get words one by one
        #add into documents our corpus
        documents.append((w,intent['tag']))
        #add our classes into list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)


[(['Merhaba'], 'greeting'), (['Nasılsın'], 'greeting'), (['Selam'], 'greeting'), (['Sa'], 'greeting'), (['İyi', 'günler'], 'greeting'), (['Görüşürüz'], 'goodbye'), (['Sonra', 'görüşürüz'], 'goodbye'), (['Seninle', 'konuşmak', 'güzeldi', ',', 'görüşürüz'], 'goodbye'), (['bye'], 'goodbye'), (['Teşekkürler'], 'thanks'), (['Teşekkür', 'ederim'], 'thanks'), (['Çok', 'yardımcı', 'oldun'], 'thanks'), (['Güzel', ',', 'teşekkürler'], 'thanks'), (['Yardım', 'ettiğin', 'için', 'teşekkürler'], 'thanks'), (['Bana', 'nasıl', 'yardımcı', 'olabilirsin', '?'], 'options'), (['Benim', 'için', 'ne', 'yapabilirsin'], 'options'), (['Bana', 'nasıl', 'yardım', 'edebilirsin', '?'], 'options'), (['Bana', 'yardım', 'etmelisin', '?'], 'options'), (['Yardımına', 'ihtiyacım', 'var'], 'options'), (['satın', 'aldığım', 'ürünü', 'nasıl', 'iade', 'edebilirim'], 'ürün_iadesi'), (['iade', 'işlemleri', 'için', 'neler', 'yapmam', 'gerekiyor', '?'], 'ürün_iadesi'), (['ürün', 'iade', 'edecektim'], 'ürün_iadesi'), (['ürün', '

In [5]:
#stem and lower each word, remove duplicates
words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))
print(len(words))
print(len(documents))
print(len(classes))
print(words)

88
42
9
[',', 'aldığım', 'almak', 'almalıyım', 'alımı', 'ban', 'bağlanmak', 'benim', 'bilg', 'bir', 'birin', 'bye', 'dağıtım', 'edebilirim', 'edebilirsin', 'edecektim', 'ederim', 'elim', 'etmek', 'etmelisin', 'ettiğin', 'fiyatı', 'gelir', 'gerekiy', 'görüşmek', 'görüşürüz', 'güne', 'günler', 'güzel', 'güzeldi', 'hakkınd', 'hang', 'hizmetler', 'hizmetlerin', 'iad', 'ihtiyacım', 'il', 'ipt', 'iptal', 'istiyor', 'için', 'işlemler', 'i̇y', 'kad', 'kargo', 'kargom', 'kargomun', 'kaç', 'konumu', 'konuşabileceğim', 'konuşmak', 'kullanmam', 'merhab', 'müşteri', 'nasıl', 'nasılsın', 'ne', 'ned', 'nel', 'ner', 'olabilirsin', 'oldun', 'sa', 'satın', 'selam', 'seninl', 'sipariş', 'siparişim', 'sonr', 'söyleyebilirsin', 'süresi', 'teşekkür', 'teşekkürl', 'ulaşır', 'var', 'yapabilirsin', 'yapmam', 'yardım', 'yardımcı', 'yardımın', 'zam', 'çok', 'çıktımı', 'özellikler', 'ürün', 'ürünü', 'ürünün', 'şirketin']


In [6]:
#create training data
training=[]
output_empty=[0]*len(classes)
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower())for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0) # create our bag of words array with 1, if word match found in current pattern
     # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag, output_row])
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
#shuffle features and convert into np.array
random.shuffle(training)
training=np.array(training)
#create train and test list 
train_x=list(training[:,0])
print(len(train_x))
train_y=list(training[:,1])
print(len(train_y))

42
42


D:\Anaconda\envs\threepointseven\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#creation of model 4 layers
model=Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),),activation='relu'))#first layer
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))#secod layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))#third layer
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))#4th and output layer
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               22784     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 5

In [9]:
#run model with sgd optimizer

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True);
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
#fit model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
9/9 [==============================] - 0s 1ms/step - loss: 2.2766 - accuracy: 0.0714
Epoch 2/200
9/9 [==============================] - 0s 2ms/step - loss: 2.1898 - accuracy: 0.0952
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 2.2023 - accuracy: 0.0714
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 2.1675 - accuracy: 0.1190
Epoch 5/200
9/9 [==============================] - 0s 2ms/step - loss: 2.1287 - accuracy: 0.1667
Epoch 6/200
9/9 [==============================] - 0s 2ms/step - loss: 2.0702 - accuracy: 0.1429
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 2.0307 - accuracy: 0.2857
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 1.9446 - accuracy: 0.2857
Epoch 9/200
9/9 [==============================] - 0s 1ms/step - loss: 1.9817 - accuracy: 0.2619
Epoch 10/200
9/9 [==============================] - 0s 1ms/step - loss: 1.9172 - accuracy: 0.2857
Epoch 11/200
9/9 [===========

9/9 [==============================] - 0s 1ms/step - loss: 0.1081 - accuracy: 0.9524
Epoch 85/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0257 - accuracy: 1.0000
Epoch 86/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0736 - accuracy: 0.9762
Epoch 87/200
9/9 [==============================] - 0s 1ms/step - loss: 0.1433 - accuracy: 0.9524
Epoch 88/200
9/9 [==============================] - 0s 997us/step - loss: 0.1064 - accuracy: 0.9762
Epoch 89/200
9/9 [==============================] - 0s 886us/step - loss: 0.0952 - accuracy: 0.9762
Epoch 90/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0469 - accuracy: 0.9762
Epoch 91/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0534 - accuracy: 0.9762
Epoch 92/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0741 - accuracy: 0.9524
Epoch 93/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0314 - accuracy: 1.0000
Epoch 94/200
9/9 [===========

9/9 [==============================] - 0s 1ms/step - loss: 0.0266 - accuracy: 1.0000
Epoch 167/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9524
Epoch 168/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0258 - accuracy: 1.0000
Epoch 169/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 170/200
9/9 [==============================] - 0s 997us/step - loss: 0.0719 - accuracy: 0.9762
Epoch 171/200
9/9 [==============================] - 0s 997us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 172/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 173/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0499 - accuracy: 0.9762
Epoch 174/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0469 - accuracy: 0.9762
Epoch 175/200
9/9 [==============================] - 0s 886us/step - loss: 0.0069 - accuracy: 1.0000
Epoch 176/200
9/9 

In [11]:
#data from user will be tokenized
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    #run tokenization
    sentence_words=clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [12]:
p=bow('Selam', words)
print(p)

found in bag: selam
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
#model saved
import tempfile
import os
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print(f'\nSaved model:{export_path}\saved_model.pb')


export_path = C:\Users\Emirhan\AppData\Local\Temp\1

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\Emirhan\AppData\Local\Temp\1\assets

Saved model:C:\Users\Emirhan\AppData\Local\Temp\1\saved_model.pb


In [14]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    print("similarity results:",results)
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    return response(return_list)

In [15]:
def response(return_list):#set response
    answer=[]
    for intent in data['intents']:
        if return_list[0][0]==intent['tag']:
            print(intent['tag'])
            return random.choice(intent['responses'])#pick one of random answers

In [17]:
classify_local('ürün hakkında bilgi istiyordum')

found in bag: ürün
found in bag: hakkınd
found in bag: bilg
similarity results: [1.1986948e-09 1.9385192e-07 7.7731259e-11 6.2376410e-12 3.7881280e-09
 1.7768538e-11 9.9999976e-01 2.7178799e-09 3.0338843e-08]
ürün_tanıtımı


'Hangi üründen bahsediyorsunuz kodunu alabilir miyim'

In [ ]:
user=input("Nasıl yardımcı olabilirim")
classify_local(user)